In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
print("Python Notebook works correctly!")

Python Notebook works correctly!


In [2]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    #Initializes the image, so that if there is no lines it doesnt cause errors.
    
    line_img = np.zeros(
        (
            img.shape[0],
            img.shape[1],
            3
        ),
        dtype=np.uint8
    )
    img = np.copy(img)
    if lines is None:
        return
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
            
    img = weighted_img(line_img,img) #Calls the weighted_img() function to combine the lane lines drawn and the original image
    
    return img

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [3]:
import os
os.listdir("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_images/")

['solidYellowCurve.jpg',
 'solidYellowLeft.jpg',
 'solidYellowCurve2.jpg',
 'solidWhiteRight.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidWhiteCurve.jpg']

In [4]:

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [12]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    #
    #image = mpimg.imread('SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_images/solidWhiteCurve.jpg')


    image_canny = canny(grayscale(image),100,200)
    #Applies the Canny tranfrom to the greyscale version of the image.

    height = image.shape[0]
    width = image.shape[1]
    #Gets the height and width of the image.
    
    vertices = [
        (0, height),
        (width / 2, height / 2),
        (width, height),
    ]
    #A list of the vertices of a triangle that has the base of the image (2 points on the bottom corners of the image)
    # and the thord point in the center of the image. this is the place you'd normmally find the desired lane lines in any video feed.
    # NOTE: doing this helps in cleaning the result in videos 1,3,4. However, it doesn't work on video 2 as the lane lines
    #are on the edges of the frame. So, this method is not perfect and needs reworking.
    

    cropped_image = region_of_interest(
        image_canny,
        np.array([vertices], np.int32),
    )
    #Crops the image as deescribed above.

  
    
    lines = cv2.HoughLinesP(
        cropped_image,
        rho=6,
        theta=np.pi / 60,
        threshold=160,
        lines=np.array([]),
        minLineLength=40,
        maxLineGap=25
    )
    #Gets the hough lines
    #Didn't use the provided function because I wanted to use the array "lines" further down, not the resulting image.
    
    if(not lines):
        lines = []
    #This is so that if there are no Hough Lines detected in the frame (Some frames in video 1 and all of video 2)
    #it sets the "lines" array to [] so that it doesnt cause errors further down.
 
    ##DRAWING 2 UNIFIED LANE LINES##

    left_line_x = []
    left_line_y = []
    right_line_x = []
    right_line_y = []
    #Sets 4 lists which are the x/y coordinates of all the Hough lines of the left/right lane
    
    if(len(lines) != 0) :
        for line in lines:
            for x1, y1, x2, y2 in line:
                slope = (y2 - y1) / (x2 - x1)
                if math.fabs(slope) < 0.5: #discards lines with slope ~0 (flat lines)
                    continue
                if slope <= 0:
                    left_line_x.extend([x1, x2])
                    left_line_y.extend([y1, y2])
                else:
                    right_line_x.extend([x1, x2])
                    right_line_y.extend([y1, y2])
    #This method checks for the slop of each lines to determine wether it belongs to left or right lane.
    #If it has a negative slope, then it's a left lane, and vice versa.
   

    if(len(left_line_x) != 0 and len(left_line_y) != 0 and len(right_line_x) != 0 and  len(right_line_y) != 0):            
    
        min_y = int(image.shape[0] * (3 / 5))
        max_y = int(image.shape[0])
        poly_left = np.poly1d(np.polyfit(
            left_line_y,
            left_line_x,
            deg=1
        ))

        left_x_start = int(poly_left(max_y))
        left_x_end = int(poly_left(min_y))

        poly_right = np.poly1d(np.polyfit(
            right_line_y,
            right_line_x,
           deg=1
        ))

        right_x_start = int(poly_right(max_y))
        right_x_end = int(poly_right(min_y))
        line_image = draw_lines(
            image,
            [[
                [left_x_start, max_y, left_x_end, min_y],
                [right_x_start, max_y, right_x_end, min_y],
            ]],
            thickness=5,
        )
    else:
        line_image = weighted_img(np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8),image)

    #This methods uses the polyfit method to form 2 unified lane lines using the 4 lists created above.
    
    return line_image 

In [8]:
white_output_1 = 'test_videos_output/1.mp4'
white_output_2 = 'test_videos_output/2.mp4'
white_output_3 = 'test_videos_output/3.mp4'
white_output_4 = 'test_videos_output/4.mp4'

## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)

clip1 = VideoFileClip("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_videos/1.mp4")
white_clip_1 = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip_1.write_videofile(white_output_1, audio=False)

clip2 = VideoFileClip("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_videos/2.mp4")
white_clip_2 = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip_2.write_videofile(white_output_2, audio=False)

clip3 = VideoFileClip("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_videos/3.mp4")
white_clip_3 = clip3.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip_3.write_videofile(white_output_3, audio=False)

clip4 = VideoFileClip("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_videos/4.mp4")
white_clip_4 = clip4.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip_4.write_videofile(white_output_4, audio=False)


[MoviePy] >>>> Building video test_videos_output/1.mp4
[MoviePy] Writing video test_videos_output/1.mp4


100%|██████████| 251/251 [00:20<00:00, 12.50it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/1.mp4 

CPU times: user 6.75 s, sys: 1.41 s, total: 8.16 s
Wall time: 22.5 s


TypeError: object of type 'NoneType' has no len()

In [13]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    #
    #image = mpimg.imread('SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_images/solidWhiteCurve.jpg')


    image_canny = canny(grayscale(image),100,200)
    #Applies the Canny tranfrom to the greyscale version of the image.

    height = image.shape[0]
    width = image.shape[1]
    #Gets the height and width of the image.
    
    vertices = [
        (0, height),
        (width / 2, height / 2),
        (width, height),
    ]
    #A list of the vertices of a triangle that has the base of the image (2 points on the bottom corners of the image)
    # and the thord point in the center of the image. this is the place you'd normmally find the desired lane lines in any video feed.
    # NOTE: doing this helps in cleaning the result in videos 1,3,4. However, it doesn't work on video 2 as the lane lines
    #are on the edges of the frame. So, this method is not perfect and needs reworking.
    

    cropped_image = region_of_interest(
        image_canny,
        np.array([vertices], np.int32),
    )
    #Crops the image as deescribed above.

  
    
    lines = cv2.HoughLinesP(
        cropped_image,
        rho=6,
        theta=np.pi / 60,
        threshold=160,
        lines=np.array([]),
        minLineLength=40,
        maxLineGap=25
    )
    #Gets the hough lines
    #Didn't use the provided function because I wanted to use the array "lines" further down, not the resulting image.
    
    if(lines is None):
        lines = []
    #This is so that if there are no Hough Lines detected in the frame (Some frames in video 1 and all of video 2)
    #it sets the "lines" array to [] so that it doesnt cause errors further down.
 
    ##DRAWING 2 UNIFIED LANE LINES##

    left_line_x = []
    left_line_y = []
    right_line_x = []
    right_line_y = []
    #Sets 4 lists which are the x/y coordinates of all the Hough lines of the left/right lane
    
    if(len(lines) != 0) :
        for line in lines:
            for x1, y1, x2, y2 in line:
                slope = (y2 - y1) / (x2 - x1)
                if math.fabs(slope) < 0.5: #discards lines with slope ~0 (flat lines)
                    continue
                if slope <= 0:
                    left_line_x.extend([x1, x2])
                    left_line_y.extend([y1, y2])
                else:
                    right_line_x.extend([x1, x2])
                    right_line_y.extend([y1, y2])
    #This method checks for the slop of each lines to determine wether it belongs to left or right lane.
    #If it has a negative slope, then it's a left lane, and vice versa.
   

    if(len(left_line_x) != 0 and len(left_line_y) != 0 and len(right_line_x) != 0 and  len(right_line_y) != 0):            
    
        min_y = int(image.shape[0] * (3 / 5))
        max_y = int(image.shape[0])
        poly_left = np.poly1d(np.polyfit(
            left_line_y,
            left_line_x,
            deg=1
        ))

        left_x_start = int(poly_left(max_y))
        left_x_end = int(poly_left(min_y))

        poly_right = np.poly1d(np.polyfit(
            right_line_y,
            right_line_x,
           deg=1
        ))

        right_x_start = int(poly_right(max_y))
        right_x_end = int(poly_right(min_y))
        line_image = draw_lines(
            image,
            [[
                [left_x_start, max_y, left_x_end, min_y],
                [right_x_start, max_y, right_x_end, min_y],
            ]],
            thickness=5,
        )
    else:
        line_image = weighted_img(np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8),image)

    #This methods uses the polyfit method to form 2 unified lane lines using the 4 lists created above.
    
    return line_image 

In [14]:
white_output_1 = 'test_videos_output/1.mp4'
white_output_2 = 'test_videos_output/2.mp4'
white_output_3 = 'test_videos_output/3.mp4'
white_output_4 = 'test_videos_output/4.mp4'

## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)

clip1 = VideoFileClip("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_videos/1.mp4")
white_clip_1 = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip_1.write_videofile(white_output_1, audio=False)

clip2 = VideoFileClip("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_videos/2.mp4")
white_clip_2 = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip_2.write_videofile(white_output_2, audio=False)

clip3 = VideoFileClip("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_videos/3.mp4")
white_clip_3 = clip3.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip_3.write_videofile(white_output_3, audio=False)

clip4 = VideoFileClip("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_videos/4.mp4")
white_clip_4 = clip4.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip_4.write_videofile(white_output_4, audio=False)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [15]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    #
    #image = mpimg.imread('SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_images/solidWhiteCurve.jpg')


    image_canny = canny(grayscale(image),100,200)
    #Applies the Canny tranfrom to the greyscale version of the image.

    height = image.shape[0]
    width = image.shape[1]
    #Gets the height and width of the image.
    
    vertices = [
        (0, height),
        (width / 2, height / 2),
        (width, height),
    ]
    #A list of the vertices of a triangle that has the base of the image (2 points on the bottom corners of the image)
    # and the thord point in the center of the image. this is the place you'd normmally find the desired lane lines in any video feed.
    # NOTE: doing this helps in cleaning the result in videos 1,3,4. However, it doesn't work on video 2 as the lane lines
    #are on the edges of the frame. So, this method is not perfect and needs reworking.
    

    cropped_image = region_of_interest(
        image_canny,
        np.array([vertices], np.int32),
    )
    #Crops the image as deescribed above.

  
    
    lines = cv2.HoughLinesP(
        cropped_image,
        rho=6,
        theta=np.pi / 60,
        threshold=160,
        lines=np.array([]),
        minLineLength=40,
        maxLineGap=25
    )
    #Gets the hough lines
    #Didn't use the provided function because I wanted to use the array "lines" further down, not the resulting image.
    
    if(lines is None):
        lines = []
    #This is so that if there are no Hough Lines detected in the frame (Some frames in video 1 and all of video 2)
    #it sets the "lines" array to [] so that it doesnt cause errors further down.
 
    ##DRAWING 2 UNIFIED LANE LINES##

    left_line_x = []
    left_line_y = []
    right_line_x = []
    right_line_y = []
    #Sets 4 lists which are the x/y coordinates of all the Hough lines of the left/right lane
    
    if(len(lines) != 0) :
        for line in lines:
            for x1, y1, x2, y2 in line:
                slope = (y2 - y1) / (x2 - x1)
                if math.fabs(slope) < 0.5: #discards lines with slope ~0 (flat lines)
                    continue
                if slope <= 0:
                    left_line_x.extend([x1, x2])
                    left_line_y.extend([y1, y2])
                else:
                    right_line_x.extend([x1, x2])
                    right_line_y.extend([y1, y2])
    #This method checks for the slop of each lines to determine wether it belongs to left or right lane.
    #If it has a negative slope, then it's a left lane, and vice versa.
   

    if(len(left_line_x) != 0 and len(left_line_y) != 0 and len(right_line_x) != 0 and  len(right_line_y) != 0):            
    
        min_y = int(image.shape[0] * (3 / 5))
        max_y = int(image.shape[0])
        poly_left = np.poly1d(np.polyfit(
            left_line_y,
            left_line_x,
            deg=1
        ))

        left_x_start = int(poly_left(max_y))
        left_x_end = int(poly_left(min_y))

        poly_right = np.poly1d(np.polyfit(
            right_line_y,
            right_line_x,
           deg=1
        ))

        right_x_start = int(poly_right(max_y))
        right_x_end = int(poly_right(min_y))
        line_image = draw_lines(
            image,
            [[
                [left_x_start, max_y, left_x_end, min_y],
                [right_x_start, max_y, right_x_end, min_y],
            ]],
            thickness=5,
        )
    else:
        line_image = weighted_img(np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8),image)

    #This methods uses the polyfit method to form 2 unified lane lines using the 4 lists created above.
    
    return line_image 

In [16]:
white_output_1 = 'test_videos_output/1.mp4'
white_output_2 = 'test_videos_output/2.mp4'
white_output_3 = 'test_videos_output/3.mp4'
white_output_4 = 'test_videos_output/4.mp4'

## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)

clip1 = VideoFileClip("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_videos/1.mp4")
white_clip_1 = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip_1.write_videofile(white_output_1, audio=False)

clip2 = VideoFileClip("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_videos/2.mp4")
white_clip_2 = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip_2.write_videofile(white_output_2, audio=False)

clip3 = VideoFileClip("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_videos/3.mp4")
white_clip_3 = clip3.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip_3.write_videofile(white_output_3, audio=False)

clip4 = VideoFileClip("SDC-Bachelor-Task-master/SDC-Bachelor-Task-master/test_videos/4.mp4")
white_clip_4 = clip4.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip_4.write_videofile(white_output_4, audio=False)


[MoviePy] >>>> Building video test_videos_output/1.mp4
[MoviePy] Writing video test_videos_output/1.mp4


100%|██████████| 251/251 [00:20<00:00, 12.04it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/1.mp4 

CPU times: user 6.75 s, sys: 1.4 s, total: 8.15 s
Wall time: 23.3 s
[MoviePy] >>>> Building video test_videos_output/2.mp4
[MoviePy] Writing video test_videos_output/2.mp4


100%|██████████| 201/201 [00:09<00:00, 20.49it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/2.mp4 

CPU times: user 3.49 s, sys: 1.09 s, total: 4.57 s
Wall time: 11 s
[MoviePy] >>>> Building video test_videos_output/3.mp4
[MoviePy] Writing video test_videos_output/3.mp4


100%|█████████▉| 221/222 [00:10<00:00, 21.99it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/3.mp4 

CPU times: user 3.09 s, sys: 768 ms, total: 3.86 s
Wall time: 11.5 s
[MoviePy] >>>> Building video test_videos_output/4.mp4
[MoviePy] Writing video test_videos_output/4.mp4


100%|█████████▉| 681/682 [00:32<00:00, 13.76it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/4.mp4 

CPU times: user 9.61 s, sys: 2.26 s, total: 11.9 s
Wall time: 34.6 s
